In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from jupyter_dash import JupyterDash

## Result Generation

Default params from the emulator source are 8, 6, 4, 2, and 8 respectively. Keep `MAXLUTS` and `MAXREGS` at their defaults for now and sweep `MAXPUSH` & `MAXAPS` from $2\rightarrow 9$ and `APSIZE` $3\rightarrow 9$.

In [ ]:
emu_cmd = lambda file, MAXPUSH, APSIZE, MAXAPS, MAXLUTS, MAXREGS :\
  f'~/Sync/haflang/reduceron/flite/dist/build/flite/flite -s -i{MAXAPS} -r{MAXPUSH}:{APSIZE}:{MAXAPS}:{MAXLUTS}:{MAXREGS} {file} | ~/Sync/haflang/tommythorn_reduceron/emulator/emu -v -'

runs = []
for maxpush in range(2,10):
    for apsize in range(3,10):
        for maxaps in range(2, 10):
            for src_file in ['orig.fl','ghc_noopt.fl', 'ghc_o2.fl']:
                cmd = emu_cmd(f'adjoxo/{src_file}',maxpush, apsize, maxaps, 2, 8)
                stats_raw = !$cmd
                stats = dict([line.replace(" ", "").split('=') for line in stats_raw[2:-1]])
                stats.update({'MAXPUSH' : maxpush, 'APSIZE' : apsize, 'MAXAPS' : maxaps, 'source': src_file})
                runs.append(stats)
                print(stats)

df = pd.DataFrame(runs)
df.to_csv('adjoxo_param_sweep.csv')

## Analysis

In [ ]:
df = pd.read_csv('adjoxo_param_sweep.csv')
df['MTicks'] = df['Ticks'].map(lambda x : float(x)/1000000.0)

In [ ]:
import plotly.graph_objects as go # or plotly.express as px

app = JupyterDash()

app.layout = html.Div([
    html.Div(
        [
            dcc.Dropdown(
                id='comparison',
                options=[{
                    'label': i,
                    'value': i
                } for i in ['MAXPUSH-MAXAPS','MAXPUSH-APSIZE','MAXAPS-APSIZE']],
                value='MAXPUSH-APSIZE',
                style={'min-width':200, 'max-width':'30%', 'margin-right':'3%'}),
            dcc.Dropdown(
                id='zkey',
                options=[{
                    'label': i,
                    'value': i
                } for i in ['MTicks','MaxHeap','MaxStack','MaxUStack','MaxLStack']],
                value='MTicks',
                style={'min-width':200, 'max-width':'30%','margin-right':'3%'}),
            dcc.Dropdown(
                id='sources',
                options=[{
                    'label': i,
                    'value': i
                } for i in ['orig.fl','ghc_noopt.fl','ghc_o2.fl']],
                value=['orig.fl'],
                multi=True,
                style={'min-width':200, 'max-width':'30%','margin-right':'3%'}),
            html.Div([dcc.RangeSlider(id='constrange', min=3, max=9, step=1, value=[3,9])],
                     style={'min-width':200, 'max-width':'30%'})
        ],
        style={'width': '100%',
               'display': 'flex'}),
    dcc.Graph(id='surfplot', figure=fig),
])


@app.callback(
    dash.dependencies.Output('surfplot', 'figure'),
    [dash.dependencies.Input('comparison', 'value'),
     dash.dependencies.Input('zkey', 'value'),
     dash.dependencies.Input('constrange', 'value'),
     dash.dependencies.Input('sources', 'value')])
def update_graph(comparison, zkey, consts_range, sources):
    (xkey,ykey) = comparison.split('-')
    constkey = list(set(['MAXPUSH','MAXAPS','APSIZE'])-set([xkey,ykey]))[0]

    planes = [ go.Surface(name=f'{constkey}@{const}',
                          showscale=False, opacity=1.0,
                          colorscale=colours,
                          x=list(set(df[xkey].values)),
                          y=list(set(df[ykey].values)),
                          z=df[(df[constkey]==const) & (df['source']==source)].pivot(index=xkey, columns=ykey, values=zkey).values)  \
               for const in range(consts_range[0],consts_range[1]+1) \
               for (source,colours) in zip(sources,colour_schemes) ]
    
    return {
        'data': planes,
        'layout': go.Layout(
            width=1000, height=800,
            autosize=True, uirevision='keep',
            scene = dict(xaxis={'title': xkey},
                         yaxis={'title': ykey},
                         zaxis={'title': zkey})
        )
    }


colour_schemes = ['Greys','YlGnBu','Greens','YlOrRd','Bluered','RdBu','Reds','Blues','Picnic']
initFig = update_graph('MAXPUSH-APSIZE', 'MTicks', [6,6], ['orig.fl'])
fig = go.Figure(layout=initFig['layout'],data=initFig['data'])
app.run_server(debug=True, use_reloader=False)

In [ ]:
df